In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import Flatten

from nltk import word_tokenize, pos_tag, chunk
from pprint import pprint
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from pprint import pprint
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [3]:
#Load cleaned dataset
data = pd.read_csv('../../Results/25_cleaned_job_descriptions.csv', header = 0, names = ['Query', 'Description'])

In [5]:
train, test = train_test_split(data, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']

In [26]:
num_labels = len(train_labels.unique().tolist())
vocab_size = 1000
batch_size = 500
nb_epoch = 30

# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_descs)
x_train = tokenizer.texts_to_matrix(train_descs, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_descs, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

In [33]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               512512    
_________________________________________________________________
activation_22 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_23 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 25)                12825     
__________

In [34]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.1)

Train on 7200 samples, validate on 800 samples
Epoch 1/30
7200/7200 [==============================] - 1s 177us/step - loss: 2.7797 - categorical_accuracy: 0.2165 - acc: 0.2165 - val_loss: 1.9880 - val_categorical_accuracy: 0.4275 - val_acc: 0.4275
Epoch 2/30
7200/7200 [==============================] - 1s 70us/step - loss: 1.5676 - categorical_accuracy: 0.5469 - acc: 0.5469 - val_loss: 1.6380 - val_categorical_accuracy: 0.5150 - val_acc: 0.5150
Epoch 3/30
7200/7200 [==============================] - 0s 69us/step - loss: 1.0079 - categorical_accuracy: 0.7072 - acc: 0.7072 - val_loss: 1.4554 - val_categorical_accuracy: 0.5712 - val_acc: 0.5712
Epoch 4/30
7200/7200 [==============================] - 1s 71us/step - loss: 0.6580 - categorical_accuracy: 0.8121 - acc: 0.8121 - val_loss: 1.4297 - val_categorical_accuracy: 0.6037 - val_acc: 0.6037
Epoch 5/30
7200/7200 [==============================] - 1s 71us/step - loss: 0.4270 - categorical_accuracy: 0.8867 - acc: 0.8867 - val_loss: 1.4142 

In [36]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('\nTest categorical_crossentropy:', score[0])
print('Categorical accuracy:', score[1])
print('Accuracy:', score[2])


2000/2000 [==============================] - 0s 25us/step

Test categorical_crossentropy: 1.7448990941047668
Categorical accuracy: 0.6319999992847443
Accuracy: 0.6319999992847443
